In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os

root_dir = '../archive/'
train_dir = root_dir + 'train/'
test_dir = root_dir + 'test/'
train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)

for item in range(len(train_files)-1):
    if train_files[item].startswith('.'):
        train_files.pop(item)
    if test_files[item].startswith('.'):
        test_files.pop(item)
        
print('train_files --',train_files)
print('test_files  --',test_files)

train_files -- ['happy', 'sad', 'fearful', 'neutral', 'angry', 'disgusted', 'surprised']
test_files  -- ['happy', 'sad', 'fearful', 'neutral', 'angry', 'disgusted', 'surprised']


In [16]:
#samples from each category of emotion and puts them in a dictionary for test and train set

def get_images(train_dir, test_dir):
    train_emo = {}
    test_emo = {}
    for emotion in train_files:
        path = train_dir + '{}/'.format(emotion)
        files = os.listdir(path)
        train_emo[emotion] = files[:10]
    for emoti in test_files:
        pth = test_dir + '{}/'.format(emoti)
        fls = os.listdir(pth)
        test_emo[emoti] = fls[:10]
    return test_emo, train_emo

def png_to_np(directory, dictionary):
    labels = dictionary.keys()
    img_nps = np.zeros((2304,), dtype=int)
    for i in labels: #for each emotion
        for image in dictionary[i]:
            print(image)
            #arr = imageio.imread(directory + i + '/'+image)
            #flat_arr = arr.flatten()
            #img_nps = np.vstack((img_nps,flat_arr))
    #img_nps = np.delete(img_nps,0,0)
    return img_nps
            

tr, ts = get_images(train_dir, test_dir) #dictionaries of image paths with labels 
train_images = png_to_np(train_dir, tr)
test_images = png_to_np(test_dir, ts)
print(train_images.shape) #70 images with 2304 features (flattened from 48x48)

im771.png
im765.png
im1209.png
im1547.png
im995.png
im1221.png
im759.png
im1235.png
im981.png
im1553.png
im771.png
im765.png
im1209.png
im995.png
im1221.png
im759.png
im1235.png
im981.png
im956.png
im942.png
im771.png
im765.png
im995.png
im759.png
im981.png
im956.png
im942.png
im573.png
im215.png
im201.png
im771.png
im765.png
im1209.png
im995.png
im1221.png
im759.png
im981.png
im956.png
im942.png
im573.png
im771.png
im765.png
im759.png
im956.png
im942.png
im573.png
im215.png
im201.png
im567.png
im229.png
im87.png
im93.png
im44.png
im50.png
im78.png
im79.png
im51.png
im45.png
im92.png
im86.png
im771.png
im765.png
im759.png
im573.png
im215.png
im201.png
im567.png
im229.png
im598.png
im407.png
im4403.png
im2072.png
im6214.png
im771.png
im6572.png
im2714.png
im4365.png
im2700.png
im4371.png
im765.png
im4403.png
im2072.png
im771.png
im2714.png
im4365.png
im2700.png
im4371.png
im765.png
im1209.png
im3378.png
im2072.png
im771.png
im2714.png
im2700.png
im765.png
im1209.png
im3378.png
im2066.pn

In [6]:
ts


{'happy': ['im4403.png',
  'im2072.png',
  'im6214.png',
  'im771.png',
  'im6572.png',
  'im2714.png',
  'im4365.png',
  'im2700.png',
  'im4371.png',
  'im765.png'],
 'sad': ['im4403.png',
  'im2072.png',
  'im771.png',
  'im2714.png',
  'im4365.png',
  'im2700.png',
  'im4371.png',
  'im765.png',
  'im1209.png',
  'im3378.png'],
 'fearful': ['im2072.png',
  'im771.png',
  'im2714.png',
  'im2700.png',
  'im765.png',
  'im1209.png',
  'im3378.png',
  'im2066.png',
  'im1547.png',
  'im3350.png'],
 'neutral': ['im4403.png',
  'im2072.png',
  'im771.png',
  'im2714.png',
  'im4365.png',
  'im2700.png',
  'im4371.png',
  'im765.png',
  'im1209.png',
  'im3378.png'],
 'angry': ['im2072.png',
  'im771.png',
  'im2714.png',
  'im2700.png',
  'im765.png',
  'im1209.png',
  'im3378.png',
  'im2066.png',
  'im1547.png',
  'im3350.png'],
 'disgusted': ['im215.png',
  'im201.png',
  'im229.png',
  'im407.png',
  'im361.png',
  'im87.png',
  'im93.png',
  'im375.png',
  'im413.png',
  'im349.png

In [7]:
test_dir

'../archive/test/'

In [13]:
la = np.zeros((2304,), dtype=int)
im = imageio.imread(train_dir+'surprised/im2714.png')

im = im.flatten()
im.shape
np.vstack((la,im))

array([[  0,   0,   0, ...,   0,   0,   0],
       [254, 255, 249, ..., 141, 151,  80]])